<a href="https://colab.research.google.com/github/samilarinc/engineerdocs/blob/main/Teknofest_Most_Generic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
task = int(input('0 for birads, 1 for kompo: '))
batch_size = int(input('Enter batch size: '))
target_names = [['BI-RADS0', 'BI-RADS1-2', 'BI-RADS4-5'], ['A', 'B', 'C', 'D']][task]

data_list = ["teknofest"]
if input("Use VinDr (n/y)") == "y":
  data_list.append("vindr")

if input("Use rsna0 (n/y)") == "y":
  data_list.append("rsna0")

model_name = input("Enter model name (timm or torch): ") #vit_base_patch16_224

0 for birads, 1 for kompo: 1
Enter batch size: 16
Use VinDr (n/y)n
Use rsna0 (n/y)n


KeyboardInterrupt: ignored

# DATA LOAD

In [ ]:
import os
first_run = 'csv_folder.zip' not in os.listdir()

In [ ]:
if first_run:
    from google.colab import drive
    drive.mount("drive")

Mounted at drive


In [ ]:
if first_run:
    os.system("pip install -q timm")

In [ ]:
if first_run: 
    os.system('cp drive/MyDrive/csv_folder.zip .')
    os.system('unzip csv_folder.zip -d .')

In [ ]:
"""    TEKNOFEST    """
if first_run:
    os.system('cp /content/drive/MyDrive/processed_teknofest.zip .')
    os.system('unzip processed_teknofest.zip -d .')

"""    RSNA_0    """
if first_run and ("rsna0" in data_list):
    os.system('cp /content/drive/MyDrive/rsna_birads0_clahe.zip .')
    os.system('unzip rsna_birads0_clahe.zip')
    os.system('mv rsna_birads0_clahe processed_rsna0')
    os.system('cp drive/MyDrive/rsna0_birads_fixed.csv .')

"""     VINDR    """
if first_run and ("vindr" in data_list):
    os.system('cp /content/drive/MyDrive/processed_vindr.zip .')
    os.system('unzip processed_vindr.zip -d .')
    os.system('rm processed_vindr/dbca9d28baa3207b3187c4d07dc81a80*')

# IMPORTS

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import os
import gc
import cv2
import time

from tqdm import tqdm
from time import perf_counter
from timm import create_model

import torch as t
from torch.utils.data import Dataset, DataLoader, random_split, ConcatDataset
import torchvision as tv
from torchvision import io
import torchvision.transforms as T
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torchsummary import summary

import pandas as pd
from PIL import Image
import warnings
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore", category=UserWarning)

device = 'cuda' if t.cuda.is_available() else 'cpu'
# !nvidia-smi

# Define Dataset

In [ ]:
class MemeDataset(Dataset):
    def __init__(self, image_root_dir, transform, csv_file, model_type = 'MEME KOMPOZİSYONU'):
        self.data = pd.read_csv(csv_file, index_col = 0)
        y = pd.get_dummies(self.data[model_type], prefix = '')
        if model_type != 'MEME KOMPOZİSYONU':
            if image_root_dir.split('_')[1] == 'rsna':
                self.data['one_hot'] = list(np.hstack((np.array(y), np.zeros((y.shape[0], 1)))))

            if image_root_dir.split('_')[1] == 'rsna0':
                self.data['one_hot'] = list(np.hstack((np.array(y), np.zeros((y.shape[0], 2)))))

            elif image_root_dir.split('_')[1] == 'vindr':
                self.data['one_hot'] = list(np.hstack((np.zeros((y.shape[0], 1)), np.array(y))))

            else:
                self.data['one_hot'] = list(np.array(y))
        else:
            self.data['one_hot'] = list(np.array(y))
        
        self.image_root_dir = image_root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if t.is_tensor(idx):
            idx = idx.tolist()

        img_path = os.path.join(self.image_root_dir,
                                str(self.data.iloc[idx, 0]))

        LCC =  io.read_image(img_path + '_LCC.png')
        LMLO = io.read_image(img_path + '_LMLO.png')
        RCC =  io.read_image(img_path + '_RCC.png')
        RMLO = io.read_image(img_path + '_RMLO.png')

        if LCC.shape[0] != 3:
          LCC = t.cat([LCC, LCC, LCC], dim=0)
          RCC = t.cat([RCC, RCC, RCC], dim=0)
          LMLO = t.cat([LMLO, LMLO, LMLO], dim=0)
          RMLO = t.cat([RMLO, RMLO, RMLO], dim=0)

        if self.transform:
            LCC = self.transform(LCC)
            RCC = self.transform(RCC)
            LMLO = self.transform(LMLO)
            RMLO = self.transform(RMLO)

        label = self.data.iloc[idx, 1]

        sample = {'LCC': LCC, 'LMLO': LMLO, 'RCC': RCC, 'RMLO': RMLO,
                  'label': label}
                  
        return sample

class CustomDataset(Dataset):
    def __init__(self, df, transform):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if t.is_tensor(idx):
            idx = idx.tolist()

        image_path = self.df.iloc[idx, -1]
        LCC =  io.read_image(image_path + '_LCC.png')
        LMLO = io.read_image(image_path + '_LMLO.png')
        RCC =  io.read_image(image_path + '_RCC.png')
        RMLO = io.read_image(image_path + '_RMLO.png')

        if LCC.shape[0] != 3:
          LCC = t.cat([LCC, LCC, LCC], dim=0)
          RCC = t.cat([RCC, RCC, RCC], dim=0)
          LMLO = t.cat([LMLO, LMLO, LMLO], dim=0)
          RMLO = t.cat([RMLO, RMLO, RMLO], dim=0)

        if self.transform:
            LCC = self.transform(LCC)
            RCC = self.transform(RCC)
            LMLO = self.transform(LMLO)
            RMLO = self.transform(RMLO)

        label = self.df.iloc[idx, 2]
        sample = {'LCC': LCC, 'LMLO': LMLO, 'RCC': RCC, 'RMLO': RMLO, 'label': label}
        return sample

# Focal Loss

In [ ]:
class FocalLoss(nn.modules.loss._WeightedLoss):
    def __init__(self, weight=None, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__(weight,reduction=reduction)
        self.gamma = gamma
        self.weight = weight #weight parameter will act as the alpha parameter to balance class weights

    def forward(self, input, target):
        ce_loss = F.cross_entropy(input, target, reduction=self.reduction, weight=self.weight)
        pt = t.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma * ce_loss).mean()
        return focal_loss

# 4HeadedBreast-Net Mimarisi

In [ ]:
class SelfAttentionFusion(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_heads):
        super().__init__()
        self.multihead_attn = nn.MultiheadAttention(input_dim, num_heads)
        self.linear = nn.Linear(input_dim * 4, hidden_dim)
        self.relu = nn.ReLU()

    def forward(self, LCC, LMLO, RCC, RMLO):
        features = t.stack((LCC, LMLO, RCC, RMLO), dim=0)
        attn_output, _ = self.multihead_attn(features, features, features)
        fused = attn_output.permute(1, 0, 2).reshape(attn_output.size(1), -1)
        fused = self.linear(fused)
        return self.relu(fused)

class DortBasliMemeNet(nn.Module):
    def __init__(self, f_extractor:str, fc:nn.Sequential):
        super().__init__()
        # try:
        #   self.LMLO_extractor = create_model(f_extractor, pretrained=True, num_classes=0)
        #   self.RMLO_extractor = create_model(f_extractor, pretrained=True, num_classes=0)
        #   self.RCC_extractor = create_model(f_extractor, pretrained=True, num_classes=0)
        #   self.LCC_extractor = create_model(f_extractor, pretrained=True, num_classes=0)
        
        # except:
        #   self.LMLO_extractor = getattr(tv.models, f_extractor)(pretrained = True)
        #   self.RMLO_extractor = getattr(tv.models, f_extractor)(pretrained = True)
        #   self.RCC_extractor =  getattr(tv.models, f_extractor)(pretrained = True)
        #   self.LCC_extractor =  getattr(tv.models, f_extractor)(pretrained = True)

        #   try:
        #     self.LCC_extractor = nn.Sequential(*tuple(self.LCC_extractor.features.children())[:])
        #     self.RCC_extractor = nn.Sequential(*tuple(self.RCC_extractor.features.children())[:])
        #     self.RMLO_extractor = nn.Sequential(*tuple(self.RMLO_extractor.features.children())[:])
        #     self.LMLO_extractor = nn.Sequential(*tuple(self.LMLO_extractor.features.children())[:])
        #   except:
        #     self.LCC_extractor = nn.Sequential(*tuple(self.LCC_extractor.children())[:-1])
        #     self.RCC_extractor = nn.Sequential(*tuple(self.RCC_extractor.children())[:-1])
        #     self.RMLO_extractor = nn.Sequential(*tuple(self.RMLO_extractor.children())[:-1])
        #     self.LMLO_extractor = nn.Sequential(*tuple(self.LMLO_extractor.children())[:-1])
        self.RCC_extractor = nn.Sequential(*tuple(getattr(tv.models, f_extractor)(pretrained = True).children()))[:-1]
        self.LCC_extractor = nn.Sequential(*tuple(getattr(tv.models, f_extractor)(pretrained = True).children()))[:-1]
        self.LMLO_extractor = nn.Sequential(*tuple(getattr(tv.models, f_extractor)(pretrained = True).children()))[:-1]
        self.RMLO_extractor = nn.Sequential(*tuple(getattr(tv.models, f_extractor)(pretrained = True).children()))[:-1]
        
        self.fuse_cnn = nn.Sequential(
                        nn.LazyConv2d(128, 1),
                        nn.ReLU(),
                        nn.Flatten()
                        )

        for param in self.RCC_extractor.parameters():
            param.requires_grad = False
        for param in self.LCC_extractor.parameters():
            param.requires_grad = False
        for param in self.RMLO_extractor.parameters():
            param.requires_grad = False
        for param in self.LMLO_extractor.parameters():
            param.requires_grad = False

        self.fc = fc

    def forward(self, X):
        LCC, LMLO, RCC, RMLO = X['LCC'].to(device), X['LMLO'].to(device), X['RCC'].to(device), X['RMLO'].to(device)
        LCC = self.LCC_extractor(LCC)
        LMLO = self.LMLO_extractor(LMLO)
        RCC = self.RCC_extractor(RCC)
        RMLO = self.RMLO_extractor(RMLO)
        
        upper = t.cat((LCC,RCC), dim = 3)
        lower = t.cat((LMLO,RMLO), dim = 3)
        fused = t.cat((upper,lower), dim = 2)
        fused = self.fuse_cnn(fused)

        return self.fc(fused)

# Dataset Initialization

In [ ]:
transform = T.Compose([
    T.RandomRotation((-30, 30)),
    T.GaussianBlur((5, 9)),
    T.RandomErasing(p = 0.5, scale = (0.01, 0.1), ratio = (0.5, 2)),
    T.ColorJitter(brightness=0.4, contrast=0.4),
    T.ConvertImageDtype(t.float),
])
transform2 = T.Compose([
    T.ConvertImageDtype(t.float),
])
tasks = ('birads', 'kompozisyon')
types = ('BIRADS KATEGORİSİ', 'MEME KOMPOZİSYONU')

train_sets, val_sets = [], []

for i in data_list:
    dataset = MemeDataset(image_root_dir=f'processed_{i}', transform=transform, csv_file = f'{i}_{tasks[task]}_fixed.csv', model_type = types[task])

    # convert data to DataFrame
    df = dataset.data
    df["image_path"] = df["HASTANO"].apply(lambda x: os.path.join(dataset.image_root_dir, str(x)))

    # split data in a stratified way
    if i == 'teknofest':
        train_df, test_val_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df[types[task]])
        val_df, test_df = train_test_split(test_val_df, test_size=0.5, random_state=42, stratify=test_val_df[types[task]])
        test_set = CustomDataset(test_df, transform2)
    
    else:
      if i == "vindr":
          df = df[df["BIRADS KATEGORİSİ"] != "BI-RADS1-2"]
      train_df, val_df = train_test_split(df, test_size=0.15, random_state=42, stratify=df[types[task]])
        
    
    train_sets.append(CustomDataset(train_df, transform))
    val_sets.append(CustomDataset(val_df, transform2))

train_set = ConcatDataset(train_sets)
val_set = ConcatDataset(val_sets)

train_loader = DataLoader(train_set, batch_size, True)
val_loader = DataLoader(val_set, batch_size, False)
test_loader = DataLoader(test_set, batch_size, False)

# Create Model

In [ ]:
num_class = train_sets[0].df['one_hot'].iloc[0].shape[0]
counts = np.zeros(num_class)
for x in train_sets:
    bi_mat = x.df["one_hot"].to_numpy()
    for i in bi_mat:
        ind = np.argmax(i)
        counts[ind] += 1
counts

array([1527., 3101., 2886.])

In [ ]:
a = np.array(counts)
b = np.sum(a) / (num_class * a)
class_weights = t.tensor(b, dtype=t.float)
class_weights

tensor([1.6403, 0.8077, 0.8679])

In [ ]:
fc_birads = nn.Sequential(
    nn.Flatten(),
    nn.LazyLinear(512),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.BatchNorm1d(512),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),
    nn.Dropout(0.2),
    nn.ReLU(),
    nn.Linear(256, 32),
    nn.Dropout(0.2),
    nn.BatchNorm1d(32),
    nn.ReLU(),
    nn.Linear(32, num_class),
    nn.LogSoftmax(1)
)

model = DortBasliMemeNet(model_name, fc_birads).to(device)

In [ ]:
tot_param = 0
for params in model.parameters():
  tot_param += params.numel()
print(f"The model has {tot_param:,} parameters.")

# Train and Validate

In [ ]:
# criterion = nn.CrossEntropyLoss(weight=class_weights).to(device)
criterion = FocalLoss(weight=class_weights).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

train_batch_size = len(train_loader) // batch_size + 1
val_batch_size = len(val_loader) // batch_size + 1
train_len = len(train_loader)
# Train the model
acc = 0

if f"{model_name}_{task}_{batch_size}_{data_list}" in os.listdir("/content/drive/MyDrive/"):
  if input("Do you want to continue training the existing model further (n/y)") == "y":
    model_state_dict = t.load(f"/content/drive/MyDrive/{model_name}_{task}_{batch_size}_{data_list}/model.pt")
    optimizer_state_dict = t.load(f"/content/drive/MyDrive/{model_name}_{task}_{batch_size}_{data_list}/optim.pt")
    criterion_state_dict = t.load(f"/content/drive/MyDrive/{model_name}_{task}_{batch_size}_{data_list}/criter.pt")

    model.load_state_dict(model_state_dict)
    optimizer.load_state_dict(optimizer_state_dict)
    criterion.load_state_dict(criterion_state_dict)

else:
  os.mkdir(f"/content/drive/MyDrive/{model_name}_{task}_{batch_size}_{data_list}")

for epoch in range(50):
    gc.collect()
    # Forward pass
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    train_correct = 0.0
    if epoch == 5:
        for param in model.RCC_extractor.parameters():
            param.requires_grad = True
        for param in model.LCC_extractor.parameters():
            param.requires_grad = True
        for param in model.RMLO_extractor.parameters():
            param.requires_grad = True
        for param in model.LMLO_extractor.parameters():
            param.requires_grad = True
    for sample in tqdm(train_loader):
        label = sample['label'].to(device)
        outputs = model(sample)
        # print(outputs.shape, label.shape)
        loss = criterion(outputs, label.float())
        train_loss += loss.item()
        
        _, train_predict = t.max(outputs, dim=1)
        train_correct += ( train_predict == t.max(label, axis = 1)[1] ).sum().item()

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_acc = train_correct / (train_len * batch_size)
    model.eval()
    val_loss = 0.0
    val_pred_list = []
    label_list = []
    
    with t.no_grad():
        for sample in tqdm(val_loader):
            label = sample['label'].to(device)
            label_list.extend(t.max(label, dim = 1)[1].to('cpu').detach().numpy())
            outputs = model(sample)
            loss = criterion(outputs, label.float())
            val_loss += loss.item()
            _, val_predict = t.max(outputs, dim=1)
            val_pred_list.extend(val_predict.to('cpu').detach().numpy())

    print('Epoch [%d/50]\n\tTrain Loss: %.4f \tValidation Loss: %.4f'%(epoch+1, train_loss/train_batch_size, val_loss/val_batch_size))
    if ((epoch+1) % 1) == 0:
        rep = classification_report(label_list, val_pred_list, target_names=target_names, output_dict = True)
        print(classification_report(label_list, val_pred_list, target_names=target_names))
        print(f"Train Accuracy at epoch {epoch+1}: {round(train_acc*100, 2)}%")
    if rep['accuracy'] > acc:
        acc = rep['accuracy']
        t.save(model.state_dict(), f"/content/drive/MyDrive/{model_name}_{task}_{batch_size}_{data_list}/model.pt")
        t.save(optimizer.state_dict(), f"/content/drive/MyDrive/{model_name}_{task}_{batch_size}_{data_list}/optim.pt")
        t.save(criterion.state_dict(), f"/content/drive/MyDrive/{model_name}_{task}_{batch_size}_{data_list}/criter.pt")
        # t.save(model, f"/content/drive/MyDrive/{model_name}_{task}_{batch_size}_{data_list}/direct_model.pt")

100%|██████████| 51/51 [00:14<00:00,  3.51it/s]


Epoch [1/50]
	Train Loss: 15.6529 	Validation Loss: 12.1897
              precision    recall  f1-score   support

    BI-RADS0       0.30      0.27      0.28       327
  BI-RADS1-2       0.42      0.76      0.54       665
  BI-RADS4-5       0.69      0.12      0.20       618

    accuracy                           0.41      1610
   macro avg       0.47      0.38      0.34      1610
weighted avg       0.50      0.41      0.36      1610

Train Accuracy at epoch 1: 41.9%


100%|██████████| 51/51 [00:13<00:00,  3.72it/s]


Epoch [2/50]
	Train Loss: 13.3820 	Validation Loss: 11.3891
              precision    recall  f1-score   support

    BI-RADS0       0.28      0.49      0.35       327
  BI-RADS1-2       0.59      0.27      0.37       665
  BI-RADS4-5       0.53      0.62      0.57       618

    accuracy                           0.45      1610
   macro avg       0.46      0.46      0.43      1610
weighted avg       0.50      0.45      0.45      1610

Train Accuracy at epoch 2: 46.69%


100%|██████████| 51/51 [00:13<00:00,  3.82it/s]


...

100%|██████████| 51/51 [00:13<00:00,  3.75it/s]


Epoch [46/50]
	Train Loss: 4.2503 	Validation Loss: 7.4044
              precision    recall  f1-score   support

    BI-RADS0       0.40      0.56      0.46       327
  BI-RADS1-2       0.71      0.59      0.64       665
  BI-RADS4-5       0.72      0.70      0.71       618

    accuracy                           0.63      1610
   macro avg       0.61      0.62      0.61      1610
weighted avg       0.65      0.63      0.63      1610

Train Accuracy at epoch 46: 74.56%


100%|██████████| 51/51 [00:14<00:00,  3.63it/s]


Epoch [47/50]
	Train Loss: 3.8408 	Validation Loss: 8.2849
              precision    recall  f1-score   support

    BI-RADS0       0.37      0.47      0.42       327
  BI-RADS1-2       0.65      0.76      0.70       665
  BI-RADS4-5       0.86      0.58      0.69       618

    accuracy                           0.63      1610
   macro avg       0.63      0.60      0.60      1610
weighted avg       0.67      0.63      0.64      1610

Train Accuracy at epoch 47: 75.29%


 20%|██        | 47/235 [01:02<04:35,  1.47s/it]

# Test

In [ ]:
criterion = FocalLoss(weight=class_weights).to(device)

model = t.load(f"/content/drive/MyDrive/{model_name}_{task}_{batch_size}_{data_list}/direct_model.pt")
criterion_state_dict = t.load(f"/content/drive/MyDrive/{model_name}_{task}_{batch_size}_{data_list}/criter.pt")
criterion.load_state_dict(criterion_state_dict)

model.eval()
test_loss = 0.0
test_pred_list = []
label_list = []

with t.no_grad():
    for sample in tqdm(test_loader):
        label = sample['label'].to(device)
        label_list.extend(t.max(label, dim = 1)[1].to('cpu').detach().numpy())
        outputs = model(sample)
        loss = criterion(outputs, label.float())
        test_loss += loss.item()
        _, test_predict = t.max(outputs, dim=1)
        test_pred_list.extend(test_predict.to('cpu').detach().numpy())

print("Test Loss: ", round(test_loss,4))
k = classification_report(label_list, test_pred_list, target_names=target_names)
print(k)